# Data Mining (CpSc 8650) Course Project  Quality Evaluation of Skull Stripped Brain MRI Images

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib
import random
import csv
from scipy import ndimage
import pandas as pd
from tensorflow import keras
import tensorflow.keras.optimizers as optimizers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras import layers
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [2]:
# load and return data 
def load_data(csv_file_path, test_size = 0.3, x_names = [], y_names = []):

    
    """
    Load training data and split it into training and validation set
    """
    print ("Loading data from: ",csv_file_path)
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(csv_file_path, names=x_names+y_names, skiprows=1)

    #yay dataframes, we can select rows and columns by their names
    X = data_df[x_names].values
    #and our steering commands as our output data
    y = data_df[y_names ].values

    return X, y

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def process_scan(path):
    
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    #Resize width, height and depth
    volume = resize_volume(volume)
    
    return volume

In [3]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [0,-5,5,-10,10,15,-15]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, labels):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, labels


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label



In [4]:
label_file_path = os.path.join(os.path.abspath(os.pardir), "dataset", "Label_file.csv")
X, y = load_data(label_file_path, x_names = ["Filename"], y_names = ["Recognizable-Facial-Feature","Brain-Feature-Loss"])

Loading data from:  D:\OneDrive - Clemson University\Clemson Courses\Data mining\Project\Project\dataset\Label_file.csv


## Displaying Image

In [7]:
from matplotlib import pyplot
model = keras.models.load_model('3d_image_classification_relu.h5')

In [8]:
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

skf = KFold(n_splits=5,shuffle=True,random_state=1)
skf.get_n_splits(X, y)
fold = 0

for train_index, test_index in skf.split(X):
    fold+=1
    print("Fold:"+str(fold))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    iteration=0

    outfileTrain = "Fold " + str(fold) + " Checkpoints.pkl"
    outfileTest = "Fold " + str(fold) + " Checkpoints test set.pkl"

    with open(outfileTrain, "rb") as f:
        [X_train_data, abc] = pickle.load(f)
    with open(outfileTest, "rb") as f:
        [X_valid_data] = pickle.load(f)
    print("Data Loaded")
    
    X_pred = model.predict(tf.expand_dims(X_train_data, axis=4))
    Face_pred = []
    Brain_pred = []
    Face_true = []
    Brain_true = []
    
    len(Face_pred)
    dataset_i = 0
    for single_prediction in X_pred:
        Face_pred.append(single_prediction[0]>0.5)
        Brain_pred.append(single_prediction[1]>0.5)
        
        Face_true.append(y_train[dataset_i][0])
        Brain_true.append(y_train[dataset_i][1])
        dataset_i+=1
    
    print(classification_report(Face_true, Face_pred))
    print(classification_report(Brain_true, Brain_pred))

Fold:1
Data Loaded
              precision    recall  f1-score   support

       False       0.99      0.99      0.99       506
        True       0.99      0.99      0.99       542

    accuracy                           0.99      1048
   macro avg       0.99      0.99      0.99      1048
weighted avg       0.99      0.99      0.99      1048

              precision    recall  f1-score   support

       False       1.00      0.97      0.99       555
        True       0.97      1.00      0.98       493

    accuracy                           0.99      1048
   macro avg       0.99      0.99      0.99      1048
weighted avg       0.99      0.99      0.99      1048

Fold:2
Data Loaded
              precision    recall  f1-score   support

       False       0.99      0.99      0.99       480
        True       0.99      0.99      0.99       569

    accuracy                           0.99      1049
   macro avg       0.99      0.99      0.99      1049
weighted avg       0.99      0.99   

### Predict with a image

In [ ]:
from tensorflow import keras
model = keras.models.load_model("3d_image_classification_relu.h5")

In [11]:
file = process_scan('../dataset/files/IXI002-Guys-0828-T1_bse_less_s5_r1.nii.gz')
file = tf.expand_dims(file, axis=0)

prediction = model.predict(file)

In [12]:
print("Face Recognizable:",prediction[0][0]>0.5)
print("Brain Feature loss:",prediction[0][1]>0.5)

Face Recognizable: True
Brain Feature loss: False


In [14]:
print("Probability of Face Recognizable",prediction[0][0])
print("Probability of Brain Feature loss:",prediction[0][1])

Probability of Face Recognizable 0.99927604
Probability of Brain Feature loss: 0.00049352646
